In [1]:
!pip install --quiet google-generativeai crewai crewai-tools



In [17]:
import os
import google.generativeai as genai
from google.colab import userdata
# Set your API key here
os.environ["GOOGLE_API_KEY"] = userdata.get("gemini_api2")
import re

gemini_api_key=userdata.get("gemini_api2")


In [5]:
import pdfplumber
import os
from crewai import Agent, Task, Crew, LLM
from google.colab import files


In [7]:
from crewai import LLM
# llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)
llm = LLM(model="gemini/gemini-2.5-flash", api_key=gemini_api_key)

In [14]:
def extract_text(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()

In [19]:
import os
import zipfile

from crewai import Agent, Task, Crew, LLM
def generate_portfolio():
  print("📄 Upload your resume PDF:")
  pdf_uploaded = files.upload()

  print("🖼️ Upload your profile photo (JPG/PNG):")
  img_uploaded = files.upload()

  # Extract file names
  pdf_path = list(pdf_uploaded.keys())[0]
  photo_filename = list(img_uploaded.keys())[0]
  profile_text = extract_text(pdf_path)


  extractor = Agent(
      role="Resume Extractor",
      goal="Extract professional details from resumes",
      backstory="Expert at extracting clean, structured data from unstructured PDFs.",
      llm=llm,
      verbose=True
  )

  builder = Agent(
      role="Portfolio Website Generator",
      goal="Create responsive HTML/CSS portfolio websites with user-uploaded photos",
      backstory="Frontend developer who always uses local image files correctly (e.g., photo.jpg)",
      llm=llm,
      verbose=True
  )
  extract_task = Task(
      description=f"""Extract structured information like name, summary, experience, education, and skills
      from this resume:\n\n{profile_text}""",
      expected_output="A structured resume summary in sections (name, title, skills, experience, education, etc.)",
      agent=extractor
  )

  builder_task = Task(
      description=f"""Using the structured resume content from the previous task, generate a clean and responsive portfolio website.

  - The user's profile image has been uploaded as `{photo_filename}`
  - Use this image in your HTML via: <img src="{photo_filename}" alt="Profile photo">
  - Do NOT use placeholders or external URLs

  Output two code blocks:
  - One for `index.html`
  - One for `style.css`
  """,
      expected_output="Two full code blocks: one for HTML, one for CSS. HTML should use the uploaded photo.",
      agent=builder,
      depends_on=[extract_task]
  )

  crew = Crew(
    agents=[extractor, builder],
    tasks=[extract_task, builder_task],
    verbose=True
  )

  result = crew.kickoff()

  # Get final raw output
  final_output = result.tasks_output[-1].raw
  os.makedirs("portfolio", exist_ok=True)

  html_match = re.search(r"```html(.*?)```", final_output, re.DOTALL)
  css_match = re.search(r"```css(.*?)```", final_output, re.DOTALL)

  html_code = html_match.group(1).strip() if html_match else "<!-- HTML not found -->"
  css_code = css_match.group(1).strip() if css_match else "/* CSS not found */"

  with open("portfolio/index.html", "w") as f:
      f.write(html_code)

  with open("portfolio/style.css", "w") as f:
      f.write(css_code)

  zip_path = "portfolio.zip"
  photo_path = f"portfolio/{photo_filename}"

  # Make sure the image is in the folder
  if not os.path.exists(photo_path):
      import shutil
      shutil.copy(photo_filename, photo_path)

  # Create ZIP with all contents of portfolio/
  with zipfile.ZipFile(zip_path, 'w') as zipf:
      for root, dirs, files_in_dir in os.walk("portfolio"):
          for file in files_in_dir:
              full_path = os.path.join(root, file)
              arcname = os.path.relpath(full_path, "portfolio")  # keeps file names clean in ZIP
              zipf.write(full_path, arcname)

  # Show what's in the zip (for debug)
  print("📦 Files included in ZIP:")
  with zipfile.ZipFile(zip_path, 'r') as z:
      print(z.namelist())

  # Trigger download
  files.download(zip_path)

In [31]:
generate_portfolio()

📄 Upload your resume PDF:


Saving RESUME (2).pdf to RESUME (2).pdf
🖼️ Upload your profile photo (JPG/PNG):


Saving IMG_20250221_181010415~2.jpg to IMG_20250221_181010415~2.jpg


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d7a618d-93f7-4da2-b5fe-d87989dcb1ae                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Extractor                                                                                        │
│                                                                                                                 │
│  Task: Extract structured information like name, summary, experience, education, and skills                     │
│          from this resume:                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  ROHINI                                                                                                         │
│                                                                                                                 │
│  SANKARI                                                                                                        │
│                                                                                                                 │
│  BYRA                                                                                                           │
│                                                                                                                 │
│  byrarohinisankari@gmail.com                                                                                    │
│                                                                                                                 │
│  |                                                                                                              │
│                                                                                                                 │
│  (+91)                                                                                                          │
│                                                                                                                 │
│  8499084617                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  github:                                                                                                        │
│  https://github.com/RohiniShankari                                                                              │
│                                                                                                                 │
│  linkedin                                                                                                       │
│  :                                                                                                              │
│  https://www .linkedin.com/in/rohini-sankari-byra-32252b274/                                                    │
│                                                                                                                 │
│  SKILLS                                                                                                         │
│  :                                                                                                              │
│                                                                                                                 │
│  programming                                           

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7b825ecd-ea56-4405-9556-b1c7a3e2f9ac                                                                     │
│  Agent: Portfolio Website Generator                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0d7a618d-93f7-4da2-b5fe-d87989dcb1ae                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```html                                                                                          │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│      <meta charset="UTF-8">                                                                                     │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">                                     │
│      <title>ROHINI SANKARI BYRA - Portfolio</title>                                                             │
│      <link rel="stylesheet" href="style.css">                                                                   │
│      <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap"                 │
│  rel="stylesheet">                                                                                              │
│  </head>                                                                                                        │
│  <body>                                                                                                         │
│      <header>                                                                                                   │
│          <div class="header-content">                                                                           │
│              <div class="profile-section">                                                                      │
│                  <img src="IMG_20250221_181010415~2.jpg" alt="Profile photo" class="profile-img">               │
│                  <div class="profile-text">                                                                     │
│                      <h1>ROHINI SANKARI BYRA</h1>                                                               │
│                      <p class="tagline">Aspiring Developer | AI Enthusiast</p>                                  │
│                  </div>                                                                                         │
│              </div>                                                                                             │
│              <div class="contact-social">                                                                       │
│                  <p class="contact-info">Email: <a                                                              │
│  href="mailto:byrarohinisankari@gmail.com">byrarohinisankari@gmail.com</a></p>                                  │
│                  <p class="contact-info">Phone: (+91) 8499084617</p>                                            │
│                  <div class="social-links">                                                                     │
│                      <a href="https://github.com/RohiniShankari" target="_blank" rel="noopener                  │
│  noreferrer">GitHub</a>                                                                                         │
│                      <a href="https://www.linkedin.com


📁 Files in 'portfolio/' before zipping:
- IMG_20250221_181010415~2.jpg
- style.css
- IMG_20250221_181010415~2 (2).jpg
- index.html
- IMG_20250221_181010415~2 (1).jpg
📦 Files in ZIP:
['IMG_20250221_181010415~2.jpg', 'style.css', 'IMG_20250221_181010415~2 (2).jpg', 'index.html', 'IMG_20250221_181010415~2 (1).jpg']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>